Did you finally train your model and predicted that huge images? Now that you have a HUGE (30k+, 40k+) pixel mask image you have to RLE encode it... and get a OOM exception!

Your problems are solved (at least I hope so). The following function is a modification of [this one](https://www.kaggle.com/lifa08/run-length-encode-and-decode).

BUT there is a trade-off: **the first and the last pixels are not encoded**. This save one `np.concatenate` that duplicates memory and may cause OOM.

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
# SRC: https://www.kaggle.com/lifa08/run-length-encode-and-decode
def rle_encode(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

def rle_decode(mask_rle, shape):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background
    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0] * shape[1], dtype=np.uint8)

    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)

In [ ]:
# New version
def rle_encode_less_memory(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    This simplified method requires first and last pixel to be zero
    '''
    pixels = img.T.flatten()
    
    # This simplified method requires first and last pixel to be zero
    pixels[0] = 0
    pixels[-1] = 0
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 2
    runs[1::2] -= runs[::2]
    
    return ' '.join(str(x) for x in runs)

Create a sample image.

In [ ]:
im = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (64, 64))
plt.imshow(im)

First we test for equal values.

In [ ]:
rle = rle_encode(im)
rle2 = rle_encode_less_memory(im)

assert rle == rle2

Add border on image.

In [ ]:
im[0] = 1
im[-1] = 1
im[:, 0] = 1
im[:, -1] = 1
plt.imshow(im)

Re-encode with each method.

In [ ]:
rle = rle_encode(im)
rle2 = rle_encode_less_memory(im)

assert rle != rle2

Decode and plot results. Note the missing pixels on top-left and bottom-right of second image.

In [ ]:
im1 = rle_decode(rle, im.shape)
im2 = rle_decode(rle2, im.shape)

plt.figure(figsize=(10, 5))
plt.subplot(121)
plt.title('Exact method')
plt.imshow(im1)

plt.subplot(122)
plt.title('Skipping first and last pixel method')
plt.imshow(im2)